# VV/VH median and standard deviation calculation


## This will require an installation of EarthEngine (ee) in your conda environment:

```
conda install -c conda-forge earthengine-api
```

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()
print(ee.__version__)

In [ ]:
"""Based on functions developed and collected by Dr. T. Smith https://github.com/tasmi/earthengine_code_snippets"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

import sys
import os
from shapely.geometry import Polygon, Point
#minx, miny ,maxx, maxy = ROI.envelope[0].bounds
minx, maxx = 14.5, 17
miny, maxy = -19.75, -18.5

aoi = Polygon([[minx, maxy], [maxx, maxy], [maxx, miny], [minx, miny]])

def gee_geometry_from_shapely(geom, crs='epsg:4326'):
    """ 
    Simple helper function to take a shapely geometry and a coordinate system and convert them to a 
    Google Earth Engine Geometry.
    """
    from shapely.geometry import mapping
    ty = geom.type
    if ty == 'Polygon':
        return ee.Geometry.Polygon(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)
    elif ty == 'Point':
        return ee.Geometry.Point(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)    
    elif ty == 'MultiPolygon':
        return ee.Geometry.MultiPolygon(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)
    
aoi_gee = gee_geometry_from_shapely(aoi)

In [ ]:
#Edge masking with high/low angle
def maskAngGT30(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.gt(30.63993))

def maskAngLT45(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.lt(45.53993)) 

def maskAngleGT40(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.gt(40))

In [ ]:
def fix_S1(ds, de, polygon, flt=True, orbit=False, direction='Ascending', platform='both'):
    if flt:
        #This is not log-scaled (raw power)
        S1 = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
    else:
        #This is log scaled (decibels)
        S1 = ee.ImageCollection('COPERNICUS/S1_GRD')
    
    S1 = S1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filterBounds(polygon)\
    .filterDate(ds, de)
    
    if orbit:
        S1 = S1.filter(ee.Filter.eq('relativeOrbitNumber_start', orbit))
    
    if direction == 'Ascending':
        data = S1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    else:
        data = S1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
        
    if not platform == 'both':
        data = data.filter(ee.Filter.eq('platform_number', platform))
    
    #Apply angle masking
    data = data.map(maskAngGT30)
    data = data.map(maskAngLT45)
    
    s1_crs = data.select('VV').first().projection()
    
    return data, s1_crs

In [ ]:
def apply_speckle_filt(collection):
    bn = collection.first().bandNames().getInfo()
    def applyfx(image):
        for b in bn:
            nat = toNatural(image.select(b)) #Convert to log scale
            filt = RefinedLee(nat) #Speckle Filter
            updated = toDB(filt) #Convert back to decibels
            image = image.addBands(updated.rename(b + '_filt'))
        return ee.Image(image)
    return collection.map(applyfx)

def toNatural(img):
    return ee.Image(10.0).pow(img.select(0).divide(10.0))

In [ ]:
def filter_s1(Ascending):
    def make_rat(image):
        rat = image.select('VV').divide(image.select('VH'))
        return rat.rename('VVdVH').set('system:time_start', image.get('system:time_start'))
    
    def make_rat_filt(image):
        rat = image.select('VV_filt').divide(image.select('VH_filt'))
        return rat.rename('VVdVH').set('system:time_start', image.get('system:time_start'))
    
    def make_dif(image):
        rat = image.select('VV').subtract(image.select('VH'))
        return rat.rename('VVminVH').set('system:time_start', image.get('system:time_start'))
                                       
    S1A_both = Ascending.select(['VV', 'VH']).sort('system:time_start')
    S1A_ratio = S1A_both.map(make_rat)
    S1A_dif = S1A_both.map(make_dif)
    
    S1A_both_focal = focal_med_filt(S1A_both)
    S1A_both_filt = apply_speckle_filt(S1A_both)
    
    S1A_ratio_focal = S1A_both_focal.map(make_rat_filt)
    S1A_ratio_focal = mask_invalid(S1A_ratio_focal, -5, 5)
        
    S1A_ratio_filt = S1A_both_filt.map(make_rat_filt)
    S1A_ratio_filt = mask_invalid(S1A_ratio_filt, -5, 5)
    
    return S1A_both, S1A_both_focal, S1A_both_filt, S1A_ratio, S1A_ratio_filt, S1A_ratio_focal

In [ ]:
def RefinedLee(img):
    '''
    Refined Lee Speckle Filter
    NOTE: img must be in natural units, i.e. not in dB!
    '''
    #Set up 3x3 kernels 
    weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
    kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)

    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    #Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0],
                              [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])

    sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)

    #Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    #Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    #And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    #Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    #duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)

    #Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).
                                                                          subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).
                                     gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).
                                     gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).
                                     gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
  
    #The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    #Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    #"collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum()) 

    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    #Calculate localNoiseVariance
    sigmaV = ee.Image(sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0]))

    #Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))
    
    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0],
                            [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

    rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)

    #Create stacks for mean and variance using the original kernels. Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    #and add the bands for rotated kernels
    #for (var i=1; i<4; i++) {
    for i in range(1,4):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).
                                     updateMask(directions.eq(2*i+1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).
                                   updateMask(directions.eq(2*i+1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).
                                     updateMask(directions.eq(2*i+2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).
                                   updateMask(directions.eq(2*i+2)))

    #"collapse" the stack into a single band image (due to masking, each pixel has just one value in it's 
    #directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    #And finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
    b = varX.divide(dir_var)

    result = ee.Image(dir_mean.add(b.multiply(img.subtract(dir_mean))))
    return result

def toDB(img):
    return ee.Image(img).log10().multiply(10.0)

def focal_med_filt(collection, radius=100):
    ''' 
    Apply a focal median filter to a selected band, with flexible radius
    '''
    bn = collection.first().bandNames().getInfo()
    
    def applyfx(image):
        for b in bn:
            sel = image.select(b)
            smoothed = sel.focal_median(radius, 'circle', 'meters')
            image = image.addBands(smoothed.rename(b + '_filt'))
        return image
    return collection.map(applyfx)

def mask_invalid(collection, minval, maxval, band=None):
    '''
    Mask all images in a collection by some min and max value
    '''
    
    if band:
        collection = collection.select(band)
    
    def apply_mask(image):
        mask1 = image.lt(maxval)
        mask2 = image.gt(minval)
        return image.updateMask(mask1).updateMask(mask2)
    return collection.map(apply_mask)

In [ ]:
ds2, de2 = '2018-10-01', '2021-04-30'

#loading, filtering, and correction
orbdict = {}
for orbit in [58, 131]:# Northern Namibia has two orbits -- 131 and 58
    Ascending, s1_crs = fix_S1(ds2, de2, aoi_gee, flt=False, orbit=orbit)
    S1A_both, S1A_both_focal, S1A_both_filt, S1A_ratio, S1A_ratio_filt, S1A_ratio_focal = filter_s1(Ascending)
    orbdict[orbit] = S1A_both_focal
    orbdict[str(orbit) + '_ratio'] = S1A_ratio_focal
    
av = orbdict['131_ratio'].reduce(ee.Reducer.median())
av_std = orbdict['131_ratio'].reduce(ee.Reducer.stdDev())

In [ ]:
# Save the VV/VH median and standard deviation to GeoTiff
task = ee.batch.Export.image.toDrive(av, 'VVdVH_med_Oct_2018_Apr_2021', scale=10, region= aoi_gee, crs= 'EPSG:4326', 
                                     maxPixels = 1e13)
task.start()              
task_std = ee.batch.Export.image.toDrive(av_std, 'VVdVH_std_Oct_2018_Apr_2021', scale=10, region= aoi_gee, 
                                         crs= 'EPSG:4326', maxPixels = 1e13)
task_std.start()              